# **MITSUI&CO. Commodity Prediction Challenge**
Develop a robust model for accurate and stable prediction of commodity prices

## Import Libraries

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## EDA

In [11]:
df_train = pd.read_csv('../data/external/mitsui-commodity-prediction-challenge/train.csv')

df_train.head()

,date_id,LME_AH_Close,LME_CA_Close,LME_PB_Close,LME_ZS_Close,JPX_Gold_Mini_Futures_Open,JPX_Gold_Rolling-Spot_Futures_Open,JPX_Gold_Standard_Futures_Open,JPX_Platinum_Mini_Futures_Open,JPX_Platinum_Standard_Futures_Open,...,FX_GBPCAD,FX_CADCHF,FX_NZDCAD,FX_NZDCHF,FX_ZAREUR,FX_NOKGBP,FX_NOKCHF,FX_ZARCHF,FX_NOKJPY,FX_ZARGBP
0,0,2264.5,7205.0,2570.0,3349.0,NaN,NaN,NaN,NaN,NaN,...,1.699987,0.776874,0.888115,0.689954,0.066653,0.090582,0.119630,0.078135,13.822740,0.059163
1,1,2228.0,7147.0,2579.0,3327.0,NaN,NaN,NaN,NaN,NaN,...,1.695279,0.778682,0.889488,0.692628,0.067354,0.091297,0.120520,0.079066,13.888146,0.059895
2,2,2250.0,7188.5,2587.0,3362.0,4684.0,4691.0,4684.0,3363.0,3367.0,...,1.692724,0.780186,0.894004,0.697490,0.067394,0.091478,0.120809,0.079287,13.983675,0.060037
3,3,2202.5,7121.0,2540.0,3354.0,4728.0,4737.0,4729.0,3430.0,3426.0,...,1.683111,0.785329,0.889439,0.698502,0.067639,0.091558,0.121021,0.079285,14.035571,0.059983
4,4,2175.0,7125.0,2604.0,3386.0,NaN,NaN,NaN,NaN,NaN,...,1.684816,0.787264,0.891042,0.701485,0.067443,0.091266,0.121055,0.078925,14.013760,0.059503


In [9]:
df_train_label = pd.read_csv('../data/external/mitsui-commodity-prediction-challenge/train_labels.csv')

df_train_label.head()

,date_id,target_0,target_1,target_2,target_3,target_4,target_5,target_6,target_7,target_8,...,target_414,target_415,target_416,target_417,target_418,target_419,target_420,target_421,target_422,target_423
0,0,0.005948,-0.002851,-0.004675,-0.000639,NaN,NaN,-0.006729,0.006066,NaN,...,NaN,0.021239,-0.005595,NaN,-0.004628,0.033793,NaN,0.038234,NaN,0.027310
1,1,0.005783,-0.024118,-0.007052,-0.018955,-0.031852,-0.019452,0.003002,-0.006876,-0.002042,...,0.003377,0.021372,-0.001517,0.012846,0.010547,0.030527,-0.000764,0.025021,0.003548,0.020940
2,2,0.001048,0.023836,-0.008934,-0.022060,NaN,NaN,0.037449,0.007658,NaN,...,-0.006712,0.009308,0.001857,-0.012761,-0.002345,0.017529,-0.005394,0.004835,-0.009075,0.001706
3,3,0.001700,-0.024618,0.011943,0.004778,NaN,NaN,-0.012519,-0.016896,NaN,...,NaN,0.036880,-0.015189,NaN,0.008118,0.001079,NaN,-0.015102,NaN,-0.033010
4,4,-0.003272,0.005234,0.006856,0.013312,0.023953,0.010681,-0.011649,0.002019,0.003897,...,NaN,0.004937,NaN,-0.006673,-0.016105,-0.004885,NaN,NaN,0.009514,NaN


In [12]:
df_target_pair = pd.read_csv('../data/external/mitsui-commodity-prediction-challenge/target_pairs.csv')

df_target_pair

,target,lag,pair
0,target_0,1,US_Stock_VT_adj_close
1,target_1,1,LME_PB_Close - US_Stock_VT_adj_close
2,target_2,1,LME_CA_Close - LME_ZS_Close
3,target_3,1,LME_AH_Close - LME_ZS_Close
4,target_4,1,LME_AH_Close - JPX_Gold_Standard_Futures_Close
...,...,...,...
419,target_419,4,FX_NOKUSD - LME_AH_Close
420,target_420,4,JPX_Gold_Standard_Futures_Close - US_Stock_RY_...
421,target_421,4,US_Stock_EWT_adj_close - LME_AH_Close
422,target_422,4,JPX_Platinum_Standard_Futures_Close - FX_NOKCHF


In [8]:
feature_list = list(df.columns)
feature_list

['date_id',
 'LME_AH_Close',
 'LME_CA_Close',
 'LME_PB_Close',
 'LME_ZS_Close',
 'JPX_Gold_Mini_Futures_Open',
 'JPX_Gold_Rolling-Spot_Futures_Open',
 'JPX_Gold_Standard_Futures_Open',
 'JPX_Platinum_Mini_Futures_Open',
 'JPX_Platinum_Standard_Futures_Open',
 'JPX_RSS3_Rubber_Futures_Open',
 'JPX_Gold_Mini_Futures_High',
 'JPX_Gold_Rolling-Spot_Futures_High',
 'JPX_Gold_Standard_Futures_High',
 'JPX_Platinum_Mini_Futures_High',
 'JPX_Platinum_Standard_Futures_High',
 'JPX_RSS3_Rubber_Futures_High',
 'JPX_Gold_Mini_Futures_Low',
 'JPX_Gold_Rolling-Spot_Futures_Low',
 'JPX_Gold_Standard_Futures_Low',
 'JPX_Platinum_Mini_Futures_Low',
 'JPX_Platinum_Standard_Futures_Low',
 'JPX_RSS3_Rubber_Futures_Low',
 'JPX_Gold_Mini_Futures_Close',
 'JPX_Gold_Rolling-Spot_Futures_Close',
 'JPX_Gold_Standard_Futures_Close',
 'JPX_Platinum_Mini_Futures_Close',
 'JPX_Platinum_Standard_Futures_Close',
 'JPX_RSS3_Rubber_Futures_Close',
 'JPX_Gold_Mini_Futures_Volume',
 'JPX_Gold_Rolling-Spot_Futures_Volume'